### Politeness prediction with ConvoKit

This notebook demonstrates how to train a simple classifier to predict the politeness level of a request by considering the politeness strategies used, as seen in the paper [A computational approach to politeness with application to social factors](https://www.cs.cornell.edu/~cristian/Politeness.html), using ConvoKit. Note that this notebook is *not* intended to reproduce the paper results: legacy code for reproducibility is available at this [repository](https://github.com/sudhof/politeness). 

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
import os

In [3]:
os.chdir('../../..')

In [4]:
import convokit
from convokit import Corpus, User, Utterance

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [5]:
from pandas import DataFrame
from typing import List, Dict, Set

#### 1: load annotated dataset

We will be using the wikipedia annotations from the [Stanford Politeness Corpus](https://www.cs.cornell.edu/~cristian/Politeness.html). 

Code below demonstrates how to convert the original CSV file into the corpus format expected by ConvoKit, but this resultant corpus can also be directly downloaded using the helper function `download("wiki-politeness-annotated")`. 

#### 2: annotate the corpus with politeness strategies

To get politeness strategies for each utterance, we will first obtain dependency parses for the utterances, and then check for strategy use. 

In [6]:
from convokit import Parser, PolitenessStrategies

ImportError: cannot import name 'Parser' from 'convokit' (/Users/calebchiam/Documents/GitHub/Cornell-Conversational-Analysis-Toolkit/convokit/__init__.py)

- adding dependency parses

In [7]:
wiki_corpus = Corpus(convokit.download("wiki-politeness-annotated"))

Dataset already exists at /Users/calebchiam/.convokit/downloads/wiki-politeness-annotated


In [8]:
annotator = Parser()
wiki_corpus = annotator.fit_transform(wiki_corpus)

NameError: name 'Parser' is not defined

- adding strategy information

In [9]:
ps = PolitenessStrategies()
wiki_corpus = ps.transform(wiki_corpus)

Below is an example of how a processed utterance now look. Dependency parses are stored in `parsed`, and politeness strategies are in `politeness_strategies`

In [10]:
wiki_corpus.get_utterance(629705)

Utterance({'id': 629705, 'user': User([('name', 'wiki_user')]), 'root': 629705, 'reply_to': None, 'timestamp': 'NOT_RECORDED', 'text': "Where did you learn English? How come you're taking on a third language?", 'meta': {'Normalized Score': -1.1200492637766977, 'Binary': -1, 'Annotations': {'A2UFD1I8ZO1V4G': 13, 'A2YFPO0N4GIS25': 9, 'AYG3MF094634L': 11, 'A38WUWONC7EXTO': 11, 'A15DM9BMKZZJQ6': 5}, 'parsed': Where did you learn English? How come you're taking on a third language?, 'politeness_strategies': {'feature_politeness_==Please==': 0, 'feature_politeness_==Please_start==': 0, 'feature_politeness_==Indirect_(btw)==': 0, 'feature_politeness_==Hedges==': 0, 'feature_politeness_==Factuality==': 0, 'feature_politeness_==Deference==': 0, 'feature_politeness_==Gratitude==': 0, 'feature_politeness_==Apologizing==': 0, 'feature_politeness_==1st_person_pl.==': 0, 'feature_politeness_==1st_person==': 0, 'feature_politeness_==1st_person_start==': 0, 'feature_politeness_==2nd_person==': 1, 'fea

You may want to save the corpus by doing `wiki_corpus.dump("wiki-politeness-annotated")` for further exploration. Note that if you do not specify a base path, data will be saved to `.convokit/saved-corpora` in your home directory by default. 

In [11]:
from convokit import Classifier

#### 3. predict politeness 

We will see how a simple classifier considering the use of politeness strategies perform. Note that this is only for demonstration, and not geared towards achieving best performance. 

(Most of the code below are adapted from [here](https://github.com/sudhof/politeness/blob/master/scripts/train_model.py))

In [12]:
import random
from sklearn import svm
from scipy.sparse import csr_matrix
from sklearn.metrics import classification_report

For this prediction task, we will only consider the polite vs. impolite group (i.e., those with "Binary" field being either +1 or -1)

In [13]:
binary_corpus = Corpus(utterances=[utt for utt in wiki_corpus.iter_utterances() if utt.meta["Binary"] != 0])

In [14]:
classifier = Classifier(obj_type="utterance", 
                        pred_feats=["politeness_strategies"], 
                        label_func=lambda utt: utt.meta['Binary'] == 1)

In [15]:
classifier.fit(binary_corpus)

We can then use this classifier to predict politeness labels for Utterances. As an example, we will use some test utterances, but you can also consider use this classifier to predict on new utterances. 

In [16]:
test_ids = [485293, 252109, 623560, 328144, 627508]

- predicting for test utterances

- to check predicted politeness label

In [22]:
pred2label = {1: "polite", 0: "impolite"}

for i, idx in enumerate(test_ids):
    print(i)
    test_utt = binary_corpus.get_utterance(idx)
    
    print("test utterance:\n{}".format(test_utt.text))
    print("------------------------")
    y_pred, y_prob = classifier.evaluate(obj=test_utt)
    
    print("Result: {}, probability estimates = {}\n".format(pred2label[y_pred[0]], y_prob[0]))

0
test utterance:
Blocked, templated.  Next?
------------------------
Result: impolite, probability estimates = [0.81661222 0.18338778]

1
test utterance:
Stephan, what did you mean by ''"Is English your native language? You seem to fill in a lot of things not said with your assumptions."'' on my talk?
------------------------
Result: impolite, probability estimates = [0.77826906 0.22173094]

2
test utterance:
I see you created a nonsense article yesterday because you were bored. If I unblock you will you disrupt more?
------------------------
Result: polite, probability estimates = [0.37926959 0.62073041]

3
test utterance:
I have no need to search the interwebs, all that matters is it offends people and is a violation of NPOV and MoS. "All Wikipedia articles and other encyclopedic content must be written from a neutral point of view (NPOV), representing fairly and without bias all significant views (that have been published by reliable sources)" - ess-eff is a bias term for a minorit

We note that this is an implementation of a politeness classifier is trained on a specific dataset (wikipedia) and on a specific binarization of politeness classes. Depending on your scenario, you might find it is preferable to directly use the politeness strategies, as exemplified in the [conversations gone awry example](https://github.com/CornellNLP/Cornell-Conversational-Analysis-Toolkit/blob/master/examples/conversations-gone-awry/Conversations_Gone_Awry_Prediction.ipynb), rather than a politeness label/score.